# Decision Tree: Classifying the Titanic Dataset Without Machine Learning Libraries

 ### 1. Introduction

### 2. Preprocessing

### 3. Impurity Measure

### 4. Splitting Data

### 5. Getting Full Tree

### 6. Recursive partitioning

### 7. Making Predictions With Pruned Tree

### 8. Submission




# 1.Introduction 

![](http://)This is my attempt at creating a decision tree without using only numpy and pandas libraries. I did not reference other people's code or approaches; rather I watched the StatQuest video on Decision Trees (https://www.youtube.com/watch?v=7VeUPuFGJHk) to understand the algorithm and took my own approach to code it. This was simply meant as a challenge and a learning exercise for me.

In [24]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import itertools
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
#train_data.head(10)

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


# 2.Preprocessing 
The age variable has missing data but all of the other numeric columns are fine. An approach is to group data by another variable and find the average age for each group and impute the average into the missing values. I will take group by the title of each person since people with similar title may have similar ages.

In [25]:
train_data["Name"] = train_data["Name"].str.split(',').str[1]
train_data["Name"] = train_data["Name"].str.split('.').str[0]
train_data["Name"] = train_data["Name"].str.strip()
x = train_data.groupby('Name').agg(['count']).index.get_level_values('Name')
x

Index(['Capt', 'Col', 'Don', 'Dr', 'Jonkheer', 'Lady', 'Major', 'Master',
       'Miss', 'Mlle', 'Mme', 'Mr', 'Mrs', 'Ms', 'Rev', 'Sir', 'the Countess'],
      dtype='object', name='Name')

In [26]:
train_data["Age"] = train_data.groupby("Name").transform(lambda x: x.fillna(x.mean()))['Age']
#changing sex to be 0 or 1 for female & male
train_data['Sex'].replace({'female':0,'male':1},inplace=True)
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,Mr,1,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,Mrs,0,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,Miss,0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,Mrs,0,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,Mr,1,35.0,0,0,373450,8.0500,NaN,S


In [27]:
train_data_tree = train_data.iloc[:,[False,False,True, False,True,True,True,True,False,True,False,False]]
train_labels_tree = train_data.iloc[:,1]
train_data_tree.describe()

,Pclass,Sex,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,2.308642,0.647587,29.754659,0.523008,0.381594,32.204208
std,0.836071,0.477990,13.277179,1.102743,0.806057,49.693429
min,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,21.773973,0.000000,0.000000,7.910400
50%,3.000000,1.000000,30.000000,0.000000,0.000000,14.454200
75%,3.000000,1.000000,35.898148,1.000000,0.000000,31.000000
max,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200


For the categorical variables, I'm transforming the columns into columns of dummy variables.

In [28]:
#Need to create dummy variable columns for the Pclass variable. The other variables are either binary or numeric
train_data_tree_dummy = pd.concat([train_data_tree,pd.get_dummies(train_data_tree['Pclass'], prefix='Pclass')],axis=1)

train_data_tree_dummy.drop(["Pclass"],axis=1,inplace=True)
sib_sp = pd.cut(train_data_tree_dummy["SibSp"], 3,labels=[0,1,2]).tolist()
parch = pd.cut(train_data_tree_dummy["Parch"], 3,labels=[0,1,2]).tolist()

train_data_tree_dummy.drop(["Parch"],axis=1,inplace=True)
train_data_tree_dummy["SibSp2"] = np.where(train_data_tree_dummy.SibSp==0,0,1)
train_data_tree_dummy.drop(["SibSp"],axis=1,inplace=True)



train_data_tree_dummy.describe()

,Sex,Age,Fare,Pclass_1,Pclass_2,Pclass_3,SibSp2
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.647587,29.754659,32.204208,0.242424,0.206510,0.551066,0.317621
std,0.477990,13.277179,49.693429,0.428790,0.405028,0.497665,0.465813
min,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,21.773973,7.910400,0.000000,0.000000,0.000000,0.000000
50%,1.000000,30.000000,14.454200,0.000000,0.000000,1.000000,0.000000
75%,1.000000,35.898148,31.000000,0.000000,0.000000,1.000000,1.000000
max,1.000000,80.000000,512.329200,1.000000,1.000000,1.000000,1.000000


# 3.Impurity 
For the measure of impurity, I'm using Gini. Lower values indicate better splits, so this function calculates Gini for each column to determine the best variable to split.

In [29]:
def gini_calc(train,labels):
    gini_list = []
    numeric_indices = []
    numeric_best_split = []
    counter = -1
    for i in train:
        counter +=1
        if train[i].dtype == "int64" or train[i].dtype =="uint8":
            split_1 = train[train[i] == 0]
            split_2 = train[train[i] == 1]
            split_1_index = split_1.index
            split_2_index = split_2.index
            labels_split_1 = labels[split_1_index]
            labels_split_2 = labels[split_2_index]

            val1 = (labels_split_1==0).sum()
            val2 = (labels_split_1==1).sum()
            val3 = (labels_split_2==0).sum()
            val4 = (labels_split_2==1).sum()

            gini_one = 1 - (val1/(val1+val2))**2 - (val2/(val1+val2))**2
            gini_two = 1 - (val3/(val3+val4))**2 - (val4/(val3+val4))**2                
            weighted_gini = (gini_one * (val1 + val2)/(len(train_labels_tree))) + (gini_two * (val3 + val4)/(len(train_labels_tree))) 
            gini_list.append(weighted_gini)
        elif train[i].dtype == "float64":
            numeric_indices.append(counter)
            numeric_gini_lst = []
            numeric_vals = np.array(train.sort_values([i])[i].reset_index(drop=True))
            averages = (numeric_vals[0:len(numeric_vals)-1] + numeric_vals[1:len(numeric_vals)])/2
            zeros = np.zeros(len(labels))
            ones = zeros + 1 
            
            for val in averages:
                vals_array = ones*val
                split_1 = train[train[i] <= vals_array]
                split_2 = train[train[i] >= vals_array]
                split_1_index = split_1.index
                split_2_index = split_2.index
                labels_split_1 = labels[split_1_index]
                labels_split_2 = labels[split_2_index]
                
                val1 = (labels_split_1==0).sum()
                val2 = (labels_split_1==1).sum()
                val3 = (labels_split_2==0).sum()
                val4 = (labels_split_2==1).sum()
                
                gini_one = 1 - (val1/(val1+val2))**2 - (val2/(val1+val2))**2
                gini_two = 1 - (val3/(val3+val4))**2 - (val4/(val3+val4))**2                
                weighted_gini = (gini_one * (val1 + val2)/(len(train_labels_tree))) + (gini_two * (val3 + val4)/(len(train_labels_tree)))     

                numeric_gini_lst.append(weighted_gini)
            
            index_min = np.argmin(numeric_gini_lst)
            numeric_best_split.append(numeric_vals[index_min])
            gini_list.append(min(numeric_gini_lst))
            
    return gini_list, numeric_best_split, numeric_indices
ginis, numeric_splits, numeric_index = gini_calc(train_data_tree_dummy,train_labels_tree)
print(ginis,numeric_splits, numeric_index)

[0.3333650003885904, 0.461906176612059, 0.4304294062855048, 0.43434842249657063, 0.4688911437727624, 0.4238751054331502, 0.4666626277654732] [6.0, 10.4625] [1, 2]


In [30]:
numeric_splits, numeric_index

([6.0, 10.4625], [1, 2])

# 4.Splitting 

Once we have a function that calculates Gini scores, we can write a function that splits data based on whichever variable has the lowest Gini. The Splitter function checks if the variable is binary or numeric and takes different operations depending on which. It then returns two split dataframes with the indices of the original dataframe in tact

In [31]:
def splitter(gini_array, data,labels, numeric_splits, numeric_index):
    """Splits a dataframe based on the best gini value determined from the gini function"""
    
    best_split = np.argmin(gini_array)
    best_gini = min(gini_array)
    best_var = data.columns[best_split]
    is_numeric = False
    tester = 0
    numeric_splitter = "Null"
    
    for i in numeric_index:
        if best_split == i:
            tester +=1
    
    if tester == 1:
        is_numeric = True
        numeric_splitter = numeric_splits[best_split]
        splitter = numeric_splits[best_split]
        combined = pd.concat([data, labels], axis=1, sort=False)
        df1 = combined[combined.iloc[:,best_split] <= splitter]
        df2 = combined[combined.iloc[:,best_split] > splitter]
        df1 = df1.drop(df1.columns[best_split], axis=1)
        df2 = df2.drop(df2.columns[best_split], axis=1)
        
        
        df1_labels = df1.iloc[:,-1]
        df1 = df1.iloc[:,:-1]

        df2_labels = df2.iloc[:,-1]
        df2 = df2.iloc[:,:-1]
        majority_class = 0
        if sum(df1_labels) > sum(df2_labels):
            majority_class = 1
        else:
            majority_class = 0
      
    else:
        combined = pd.concat([data, labels], axis=1, sort=False)

        df1 = combined[combined.iloc[:,best_split] == 0]
        df2 = combined[combined.iloc[:,best_split] == 1]
        df1 = df1.drop(df1.columns[best_split], axis=1)
        df2 = df2.drop(df2.columns[best_split], axis=1)   


        df1_labels = df1.iloc[:,-1]
        df1 = df1.iloc[:,:-1]

        df2_labels = df2.iloc[:,-1]
        df2 = df2.iloc[:,:-1]
        
        if sum(df1_labels) > sum(df2_labels):
            majority_class = 1
        else:
            majority_class = 0
    

    return df1, df2, df1_labels, df2_labels, best_gini, best_var, is_numeric, numeric_splitter, majority_class

# 5.Recursion 

The next two functions basically do recursion to iteratively go through the data splitting each split to a specified amount. I am specifiying a max depth of 5 but this parameter is changeable (anything from 1-6 should work). I took a list approach to contain all of the information needed to develop the set of rules that are determined later.

Note: The approach I used basically does the following: The main dataframe would be index 0. That splits into two dataframes with indices 1 and 2. Dataframe 1 then splits into two dataframes with indices 3 and 4 while Dataframe two splits into 2 dataframes with indices 5 and 6. This continues for the max depth. So there will be a full tree, then I determined which splits were actually less informative than the previous ones and created rules based on this, collecting the indices of the dataframes that actually reduced Gini.[](http://)

In [32]:
def decision_tree(data,labels):
    """Takes a dataframe and labels and applied the Gini and splitter functions to it"""
    ginis, numeric_splits, numeric_index = gini_calc(data,labels)
    df_1, df_2, labs_one, labs_two, best_gini, best_var, is_numeric, numeric_splitter, majority_class = splitter(ginis,data,labels, numeric_splits, numeric_index)
    

    return best_gini, df_1, df_2, labs_one, labs_two, best_var, is_numeric, numeric_splitter, majority_class


best_gini, df_1, df_2, labs_one, labs_two, best_var, is_numeric, numeric_splitter,majority_class = decision_tree(train_data_tree_dummy,train_labels_tree)



In [33]:
def recursive_tree(data,labels,max_depth = 5):
    """Function that takes original data and labels and iteratively splits each dataframe for the full max-depth"""
    
    best_gini, df_1, df_2, labs_one, labs_two, best_var, is_numeric, numeric_splitter, majority_class = decision_tree(data,labels)
    
    data_frame_splits = []
    ginis = []
    labs_list = []
    best_var_list = []
    is_numeric_lst = []
    numer_splitter_lst = []
    majority_class_list = []
    data_frame_splits.append(data)
    ginis.append(.5)
    labs_list.append(labels)
    best_var_list.append("NA")
    is_numeric_lst.append("NA")
    numer_splitter_lst.append("NA")
    majority_class_list.append("NA")
    
    counter = 0
    for i in range(max_depth*2):
        if counter == 0:
            best_gini, df_1, df_2, labs_one, labs_two, best_var, is_numeric, numeric_splitter, majority_class = decision_tree(data,labels)
        else:
            best_gini, df_1, df_2, labs_one, labs_two, best_var, is_numeric, numeric_splitter,majority_class = decision_tree(data_frame_splits[counter],labs_list[counter])
        counter +=1
        data_frame_splits.append(df_1)
        data_frame_splits.append(df_2)
        ginis.append(best_gini)
        ginis.append(best_gini)
        labs_list.append(labs_one)
        labs_list.append(labs_two)
        best_var_list.append(best_var)
        best_var_list.append(best_var)
        is_numeric_lst.append(is_numeric)
        is_numeric_lst.append(is_numeric)
        numer_splitter_lst.append(numeric_splitter)
        numer_splitter_lst.append(numeric_splitter)
        majority_class_list.append(majority_class)
        if majority_class == 1:
            majority_class_list.append(majority_class-1)
        else:
            majority_class_list.append(majority_class+1)
        
    return data_frame_splits, ginis,labs_list,best_var_list, is_numeric_lst,numer_splitter_lst,majority_class_list

data_frame_splits, ginis,labs_list,best_var_list, is_numeric_lst,numer_splitter_lst,majority_class_list = recursive_tree(train_data_tree_dummy,train_labels_tree)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in long_scalars
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in long_scalars
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in long_scalars
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in long_scalars


# 6.Pruning 
Now that I have the full tree, I want to find the indices of the dataframes that make up the best tree


In [34]:
def create_rules(ginis, is_numeric_lst):
    """Returns indices of best tree as well as relavent information indexed by these best indices"""
    counter = 0
    include_list = []
    for i in range(len(ginis)):
        if i == 0:
            include_list.append(counter)
        
        elif is_numeric_lst[i] == False and counter % 2 != 0:
            if ginis[i] <= ginis[int((counter-1)/2)]:
                include_list.append(counter)
        elif is_numeric_lst[i] == False and counter % 2 == 0:
            if ginis[i] <= ginis[int((counter-2)/2)]:
                 include_list.append(counter)
        elif is_numeric_lst[i] == True and counter % 2 != 0:
            if ginis[i] <= ginis[int((counter-1)/2)]:
                include_list.append(counter)
        elif is_numeric_lst[i] == True and counter % 2 == 0:
            if ginis[i] <= ginis[int((counter-2)/2)]:
                include_list.append(counter)
        counter +=1
    
    best_variables = []
    for i in include_list:
        best_variables.append(best_var_list[i])
    
    is_numeric_final = []
    for i in include_list:
        is_numeric_final.append(is_numeric_lst[i])
    
    majority_class_list_final = []
    for i in include_list:
        majority_class_list_final.append(majority_class_list[i])
    
    numer_splitter_lst_final = []
    for i in include_list:
        numer_splitter_lst_final.append(numer_splitter_lst[i])
    
    
    return include_list, is_numeric_final, best_variables, majority_class_list_final, numer_splitter_lst_final
    
inclusions, numeric_final, best_vars, majority_class_lst, numer_splitter_lst_final = create_rules(ginis, is_numeric_lst)
print(inclusions)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


# 7.Predictions 
The next function goes through the data with all of the information determined by the rules of the best tree and sets a labels column equal to value determined by the tree.



In [35]:
def yarf(data,is_numeric_final,best_variables, inclusions,majority_class_list_final,numer_splitter_lst_final, counter):
    """yet another recursive function... go through the data and set a column equal to a value determined by the rules"""
    if is_numeric_final[counter] == False:
        x = data[data[best_variables[counter]] == 0].copy()
        if majority_class_list_final[counter] == 1:
            x["Survived"] = 0
        elif majority_class_list_final[counter] == 0:
            x["Survived"] = 1
            
        y = data[data[best_variables[counter]] == 1].copy()
        if majority_class_list_final[counter] == 0:
            y["Survived"] = 0
        elif majority_class_list_final[counter] == 1:
            y["Survived"] = 1
        return x, y
        
    elif is_numeric_final[counter] == True:
        x = data[data[best_variables[counter]] <= numer_splitter_lst_final[counter]].copy()
        if majority_class_list_final[counter] == 1:
            x["Survived"] = 0
        elif majority_class_list_final[counter] == 0:
            x["Survived"] = 1
        y = data[data[best_variables[counter]] > numer_splitter_lst_final[counter]].copy()
        if majority_class_list_final[counter] == 0:
            y["Survived"] = 0
        elif majority_class_list_final[counter] == 1:
            y["Survived"] = 1
        return x, y
    



In [36]:
def recursive_through_yarf(data,is_numeric_final,best_variables, inclusions,majority_class_list_final,numer_splitter_lst_final, max_depth = 5):
    """Using the yarf function, iteratively save the dataframes with the predictions as a column"""
    x, y = yarf(data,is_numeric_final,best_variables, inclusions,majority_class_list_final,numer_splitter_lst_final,1)
    
    list_of_data_frames = []
    list_of_data_frames.append(x)
    list_of_data_frames.append(y)
    counter = 2
    frames_counter = 0
    
    for i in range(max_depth):
        x, y = yarf(list_of_data_frames[frames_counter],is_numeric_final,best_variables, inclusions,majority_class_list_final,numer_splitter_lst_final,counter)
        list_of_data_frames.append(x)
        list_of_data_frames.append(y)
        counter+=1
        frames_counter +=1
        x, y = yarf(list_of_data_frames[frames_counter],is_numeric_final,best_variables, inclusions,majority_class_list_final,numer_splitter_lst_final,counter)
        list_of_data_frames.append(x)
        list_of_data_frames.append(y)
        counter+=1
        frames_counter +=1
    
    return list_of_data_frames


#testing = recursive_through_yarf(train_data_tree_dummy,is_numeric_final,best_variables, inclusions,majority_class_list_final,numer_splitter_lst_final,max_depth = 3)
    


Below I am just creating the testing data so it is in the same format as the training data.

In [37]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data["Name"] = test_data["Name"].str.split(',').str[1]
test_data["Name"] = test_data["Name"].str.split('.').str[0]
test_data["Name"] = test_data["Name"].str.strip()
test_data["Age"] = test_data.groupby("Name").transform(lambda x: x.fillna(x.mean()))['Age']
#changing sex to be 0 or 1 for female & male
test_data['Sex'].replace({'female':0,'male':1},inplace=True)
test_data_tree = test_data.iloc[:,[False,True,False, True,True,True,True,False,True,False,False]]
test_data_tree.head()

test_data_tree_dummy = pd.concat([test_data_tree,pd.get_dummies(test_data_tree['Pclass'], prefix='Pclass')],axis=1)

test_data_tree_dummy.drop(["Pclass"],axis=1,inplace=True)
sib_sp = pd.cut(test_data_tree_dummy["SibSp"], 3,labels=[0,1,2]).tolist()
parch = pd.cut(test_data_tree_dummy["Parch"], 3,labels=[0,1,2]).tolist()
test_data_tree_dummy["SibSp2"] = np.where(test_data_tree_dummy.SibSp==0,0,1)

test_data_tree_dummy.drop(["Parch"],axis=1,inplace=True)
test_data_tree_dummy.drop(["SibSp"],axis=1,inplace=True)
test_data_tree_dummy.head()


,Sex,Age,Fare,Pclass_1,Pclass_2,Pclass_3,SibSp2
0,1,34.5,7.8292,0,0,1,0
1,0,47.0,7.0000,0,0,1,1
2,1,62.0,9.6875,0,1,0,0
3,1,27.0,8.6625,0,0,1,0
4,0,22.0,12.2875,0,0,1,1


In [38]:
testing = recursive_through_yarf(test_data_tree_dummy,numeric_final,best_vars, inclusions,majority_class_lst,numer_splitter_lst_final,max_depth = 5)


In [39]:
def make_predictions(testing_data, inclusions):
    """When merging dataframes, only want to merge leaf nodes. This function takes all the leaf nodes and merges together"""
    
    for i in inclusions:
        if i %2 !=0:
            try:
                inclusions.remove(int((i-1)/2))
            except:
                pass
        elif i %2 ==0:
            try:
                inclusions.remove(int((i-2)/2))
            except:
                pass
    inclusions2 = []
    for i in inclusions:
        inclusions2.append(i-1)
        
    dataframes_to_keep = []
    for i in inclusions2:
        dataframes_to_keep.append(testing_data[i])


    preds = pd.concat(dataframes_to_keep, axis=0).sort_index(axis = 0)

    return preds
    
preds = make_predictions(testing, inclusions)

In [40]:
preds.shape

(418, 8)

In [41]:
test_data.shape

(418, 11)

In [42]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,Mr,1,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,Mrs,0,47.0,1,0,363272,7.0000,NaN,S
2,894,2,Mr,1,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,Mr,1,27.0,0,0,315154,8.6625,NaN,S
4,896,3,Mrs,0,22.0,1,1,3101298,12.2875,NaN,S


# 8.Submission 

In [43]:
data = {'PassengerId': test_data["PassengerId"].values, 'Survived':preds["Survived"].values}

df_submission = pd.DataFrame(data)

df_submission.to_csv("submission_decision_tree.csv",index=False)

Final score on testing Set: 0.76076